In [1]:
import pandas as pd
from utils import *

NKI_submission = "/Users/tushar/CancerModels/submission/NKI/IDC"
NKI_dir = "/Users/tushar/CancerModels/pdxfinder-data/data/UPDOG/NKI"
template = "/Users/tushar/CancerModels/pdxfinder-data/template/active_templates/"

## Mutation VCF

In [ ]:
panelseq_columns = ["CHROM", "POS", "ID", "REF", "ALT", "QUAL", "FILTER", "INFO", "FORMAT", "IDC025PanelSeqPDX", "IDC026PanelSeqPDX", "IDC029PanelSeqPDX", "IDC031PanelSeqPDX", "IDC032PanelSeqPDX", "IDC038PPanelSeqDX", "IDC057PanelSeqPDX", "IDC062PanelSeqPDX", "IDC065PanelSeqPDX", "IDC069PanelSeqPDX", "IDC072PanelSeqPDX", "IDC090BPanelSeqPDX", "IDC092PanelSeqPDX", "IDC097PanelSeqPDX", "IDC099PanelSeqPDX", "IDC107PanelSeqPDX", "IDC113PanelSeqPDX", "IDC117PanelSeqPDX", "IDC143PanelSeqPDX", "IDC152PanelSeqPDX", "IDC159APanelSeqPDX", "IDC159BPanelSeqPDX", "IDC180PanelSeqPDX", "IDC186PanelSeqPDX", "IDC192PanelSeqPDX", "IDC197PanelSeqPDX", "IDC198PanelSeqPDX", "IDC207PanelSeqPDX", "IDC209PanelSeqPDX", "IDC216PanelSeqPDX", "IDC218PanelSeqPDX", "IDC222PanelSeqPDX", "IDC223BPanelSeqPDX", "IDC229PanelSeqPDX", "IDC232PanelSeqPDX", "IDC274PanelSeqPDX", "IDC282PanelSeqPDX", "IDC290PanelSeqPDX", "IDC299PanelSeqPDX", "IDC307PanelSeqPDX", "IDC338PanelSeqPDX", "IDC344PanelSeqPDX", "IDC346PanelSeqPDX", "ILC006PanelSeqPDX", "ILC012PanelSeqPDX", "ILC083PanelSeqPDX", "ILC248PanelSeqPDX"]
mutation_raw_panel_seq = pd.read_csv(join(NKI_submission, 'cm_idc_nki_panelseq_newnames.vcf'),sep='\t', comment='#',low_memory=False, names=panelseq_columns, header=None)

In [ ]:
mutation_raw_panel_seq = mutation_raw_panel_seq.replace('./.:.:.:.:.:.:.:.:.:.', '').replace('./.:.:.:.:.:.:.', '')
mutation_raw_panel_seq

In [ ]:
mutation_melted = mutation_raw_panel_seq[['CHROM', 'POS', 'ID', 'REF', 'ALT', "IDC025PanelSeqPDX", "IDC026PanelSeqPDX", "IDC029PanelSeqPDX", "IDC031PanelSeqPDX", "IDC032PanelSeqPDX", "IDC038PPanelSeqDX", "IDC057PanelSeqPDX", "IDC062PanelSeqPDX", "IDC065PanelSeqPDX", "IDC069PanelSeqPDX", "IDC072PanelSeqPDX", "IDC090BPanelSeqPDX", "IDC092PanelSeqPDX", "IDC097PanelSeqPDX", "IDC099PanelSeqPDX", "IDC107PanelSeqPDX", "IDC113PanelSeqPDX", "IDC117PanelSeqPDX", "IDC143PanelSeqPDX", "IDC152PanelSeqPDX", "IDC159APanelSeqPDX", "IDC159BPanelSeqPDX", "IDC180PanelSeqPDX", "IDC186PanelSeqPDX", "IDC192PanelSeqPDX", "IDC197PanelSeqPDX", "IDC198PanelSeqPDX", "IDC207PanelSeqPDX", "IDC209PanelSeqPDX", "IDC216PanelSeqPDX", "IDC218PanelSeqPDX", "IDC222PanelSeqPDX", "IDC223BPanelSeqPDX", "IDC229PanelSeqPDX", "IDC232PanelSeqPDX", "IDC274PanelSeqPDX", "IDC282PanelSeqPDX", "IDC290PanelSeqPDX", "IDC299PanelSeqPDX", "IDC307PanelSeqPDX", "IDC338PanelSeqPDX", "IDC344PanelSeqPDX", "IDC346PanelSeqPDX", "ILC006PanelSeqPDX", "ILC012PanelSeqPDX", "ILC083PanelSeqPDX", "ILC248PanelSeqPDX"]]
dcis_columns = [col for col in mutation_melted.columns if 'DCIS' in col]
mutation_melted = pd.melt(mutation_melted, id_vars=['CHROM', 'POS', 'ID', 'REF', 'ALT'], value_vars=dcis_columns, var_name='sample_id', value_name='value')
mutation_melted = mutation_melted[mutation_melted['value'] != '']
column_mapper = {'CHROM': 'chromosome', 'POS': 'seq_start_position', 'REF': 'ref_allele', 'ALT':'alt_allele'}
mutation_melted.rename(columns=column_mapper, inplace=True)
mutation_template = read_metadata_with_fields(join(template, 'mut/mutation_template_internal-Sheet1.tsv'))
mutation_melted['strand'] = '+'
mutation_melted['platform_id'] = 'mutation_panel_seq'
chromosomes = ['1', '10', '11', '12', '13', '14', '15', '16', '17', '18', '19', '2', '20', '21', '22', '3', '4', '5', '6', '7', '8', '9', 'X', 'Y']
mutation_melted = mutation_melted[mutation_melted['chromosome'].isin(chromosomes)]
for col in mutation_template.columns:
    if col not in mutation_melted:
        mutation_melted[col] = ''

In [ ]:
submission_mms = read_metadata_with_fields(join(NKI_dir, 'NKI_molecular_metadata-sample.tsv'))
submission_model_sample_dict = submission_mms[['model_id', 'sample_id']].drop_duplicates().groupby(['model_id'])['sample_id'].apply(set).to_dict()

In [ ]:
sample_ids = ["IDC025PanelSeqPDX", "IDC026PanelSeqPDX", "IDC029PanelSeqPDX", "IDC031PanelSeqPDX", "IDC032PanelSeqPDX", "IDC038PPanelSeqDX", "IDC057PanelSeqPDX", "IDC062PanelSeqPDX", "IDC065PanelSeqPDX", "IDC069PanelSeqPDX", "IDC072PanelSeqPDX", "IDC090BPanelSeqPDX", "IDC092PanelSeqPDX", "IDC097PanelSeqPDX", "IDC099PanelSeqPDX", "IDC107PanelSeqPDX", "IDC113PanelSeqPDX", "IDC117PanelSeqPDX", "IDC143PanelSeqPDX", "IDC152PanelSeqPDX", "IDC159APanelSeqPDX", "IDC159BPanelSeqPDX", "IDC180PanelSeqPDX", "IDC186PanelSeqPDX", "IDC192PanelSeqPDX", "IDC197PanelSeqPDX", "IDC198PanelSeqPDX", "IDC207PanelSeqPDX", "IDC209PanelSeqPDX", "IDC216PanelSeqPDX", "IDC218PanelSeqPDX", "IDC222PanelSeqPDX", "IDC223BPanelSeqPDX", "IDC229PanelSeqPDX", "IDC232PanelSeqPDX", "IDC274PanelSeqPDX", "IDC282PanelSeqPDX", "IDC290PanelSeqPDX", "IDC299PanelSeqPDX", "IDC307PanelSeqPDX", "IDC338PanelSeqPDX", "IDC344PanelSeqPDX", "IDC346PanelSeqPDX", "ILC006PanelSeqPDX", "ILC012PanelSeqPDX", "ILC083PanelSeqPDX", "ILC248PanelSeqPDX"]

for mid in submission_model_sample_dict.keys():
    temp = mutation_melted[[sid in submission_model_sample_dict[mid] for sid in mutation_melted['sample_id']]]
    #model_id = [mid for mid in submission_model_sample_dict if s in submission_model_sample_dict[mid]][0]
    #temp = mutation_melted[mutation_melted['sample_id'] == s]
    temp[mutation_template.columns].to_csv(join(NKI_submission, f'mut/NKI_mut_{mid}.tsv'), sep='\t', index=False)

## CNA data

In [ ]:
from utils import *

NKI_submission = "/Users/tushar/CancerModels/submission/NKI/IDC"
NKI_dir = "/Users/tushar/CancerModels/pdxfinder-data/data/UPDOG/NKI"
template = "/Users/tushar/CancerModels/pdxfinder-data/template/active_templates/"


def get_location_from_synonym(row):
    symbol = row['symbol']
    if isinstance(symbol, str):
        pattern = "(^|\|)" + symbol + "($|\|)"
        match = geneSymbol_location.loc[geneSymbol_location.Synonyms.str.contains(pattern)].reset_index(drop=True)
        if len(match) > 1:
            match = match.iloc[:1]
        if len(match) > 0:
            row['chromosome'], row['strand'], row['seq_start_position'], row['seq_end_position'], row['ncbi_gene_id'], \
            row['ensembl_gene_id'] = match['chromosome'][0], match['strand'][0], match['start'][0], match['end'][0], \
            match['GeneID'][0], match['ensembl_id'][0]
    return row


import requests
import re


def fetch_location_from_ucsc(url):
    chr, location = None, None
    try:
        response = requests.get(url)
        if response.ok:
            response = response.json()
            if "positionMatches" in response.keys() and 'matches' in response["positionMatches"][
                0].keys() and "position" in response["positionMatches"][0]['matches'][0].keys():
                response = response["positionMatches"][0]['matches'][0]['position']
                chr, location = response.split(':')
                location = location.split('-')
        return chr, location
    except Exception as e:
        print(f"failed for symbol {url}")
        return None, None


def get_location_from_ucsc_api(row):
    if row['chromosome'] != '':
        return row
    url = f"https://api.genome.ucsc.edu/search?search={row['symbol']}&genome=hg38"
    chr, location = fetch_location_from_ucsc(url)
    if chr is not None and location is not None:
        row['chromosome'] = chr.replace('chr', '')
        row['seq_start_position'] = int(location[0])
        row['seq_end_position'] = int(location[1])
    if chr is None or location is None:
        symbol = re.sub('\.\d*', '', row['symbol'])
        url = f"https://api.genome.ucsc.edu/search?search={symbol}&genome=hg38"
        chr, location = fetch_location_from_ucsc(url)
        if chr is not None and location is not None:
            row['chromosome'] = chr.replace('chr', '')
            row['seq_start_position'] = int(location[0])
            row['seq_end_position'] = int(location[1])
    return row


import numpy as np
from tqdm import tqdm

tqdm.pandas()


# Define a function to convert log2 ratio to fold change
def log2_to_fold_change(log2_ratio):
    return 2 ** log2_ratio


# Define a function to convert log2 ratio to log10 ratio
def log2_to_log10(log2_ratio):
    return log2_ratio / np.log10(2)


genes = pd.read_csv('../resources/genes.tsv', sep='\t')
cna_matrix = read_metadata_with_fields(join(NKI_submission, 'cna/cna_cm_dcis_nki.txt'))
cna_long_format = cna_matrix.melt(id_vars=["index"], var_name="sample_id", value_name="log2r_cna").rename(
    columns={"index": "symbol"})
cna_long_format['log10r_cna'] = cna_long_format['log2r_cna'].apply(log2_to_log10)
cna_long_format['fold_change'] = cna_long_format['log2r_cna'].apply(log2_to_fold_change)
cna_long_format = cna_long_format.merge(
    genes[['Symbol', 'chromosome', 'strand', 'start', 'end', 'GeneID', 'ensembl_id']], left_on='symbol',
    right_on='Symbol', how='left').rename(
    columns={'start': 'seq_start_position', 'end': 'seq_end_position', 'ensembl_id': 'ensembl_gene_id',
             'GeneID': 'ncbi_gene_id'})
#cna_long_format = cna_long_format.apply(get_location_from_synonym, axis=1)
cna_long_format[cna_long_format['chromosome'].isna()].drop_duplicates('symbol')
mapping = cna_long_format[['symbol', 'chromosome', 'strand', 'seq_start_position', 'seq_end_position', 'ncbi_gene_id',
                           'ensembl_gene_id']].drop_duplicates().progress_apply(get_location_from_synonym, axis=1)
other_mapping = mapping[mapping['chromosome'].isna()]
other_mapping = other_mapping.fillna('').progress_apply(get_location_from_ucsc_api, axis=1)
other_mapping = other_mapping.fillna('').progress_apply(get_location_from_ucsc_api, axis=1)
other_mapping['strand'] = '+'
mapping = pd.concat([other_mapping, mapping]).drop_duplicates(subset='symbol').reset_index(drop=True)
# Perform an outer merge
merged_df = pd.merge(cna_long_format, mapping, on=['symbol'], how='left', suffixes=('_df1', '_df2'))

In [ ]:
cols = ['chromosome', 'strand', 'seq_start_position', 'seq_end_position', 'ncbi_gene_id', 'ensembl_gene_id']
# Fill missing values in df1 columns with corresponding df2 columns
for column in cols:
    merged_df[column] = merged_df[f'{column}_df1'].combine_first(merged_df[f'{column}_df2'])

# Drop the temporary columns used for merging
merged_df.drop(columns=[f'{col}_df1' for col in cols], inplace=True)
merged_df.drop(columns=[f'{col}_df2' for col in cols], inplace=True)
merged_df.strand = merged_df.strand.astype(str).str.replace('-1.0', '-').str.replace('1.0', '+')
merged_df.strand.unique()
merged_df[merged_df['chromosome'] == ''].drop_duplicates(subset=['symbol'])
merged_df[merged_df['chromosome'] == ''].shape[0] / merged_df.shape[0] * 100
cna_long_format = merged_df[merged_df['chromosome'] != '']
mapping[(mapping['chromosome'] != '') & (mapping['strand'] == '')]
mapping


In [ ]:
submission_mms = read_metadata_with_fields(join(NKI_submission, 'molecular data/NKI_molecular_metadata_DCIS-sample.tsv'))
submission_model_sample_dict = submission_mms[['model_id', 'sample_id']].drop_duplicates().groupby(['model_id'])['sample_id'].apply(set).to_dict()

cna_template = read_metadata_with_fields(join(template, 'cna/cna_template-sheet.tsv'))
cna_long_format = cna_long_format.merge(submission_mms[['sample_id', 'platform_id']], on='sample_id', how='left')
for col in cna_template.columns:
    if col not in cna_long_format.columns:
        cna_long_format[col] = ''
cna_long_format['platform_id'] = 'cna_' + cna_long_format['platform_id']
for mid in submission_model_sample_dict.keys():
    temp = cna_long_format[[sid in submission_model_sample_dict[mid] for sid in cna_long_format['sample_id']]]
    temp[cna_template.columns].to_csv(join(NKI_submission, f'cna/NKI_cna_{mid}.tsv'), sep='\t', index=False)

## Expression

In [13]:
NKI_mms = join(NKI_dir, 'NKI_molecular_metadata-sample.tsv')
NKI_mms = read_metadata_without_fields(NKI_mms)
NKI_exp = read_metadata_with_fields(join(NKI_dir, 'expression/NKI_expression.tsv'))
NKI_exp['strand'] = NKI_exp['strand'].str.replace('plus', '+').str.replace('minus', '-')

In [53]:
submission_mms = read_metadata_with_fields(join(NKI_submission, 'molecular data/NKI_molecular_metadata_DCIS-sample.tsv'))
submission_model_sample_dict = submission_mms[['model_id', 'sample_id']].drop_duplicates().groupby(['model_id'])['sample_id'].apply(set).to_dict()
expression_matrix = read_metadata_with_fields(join(NKI_submission, 'expression/tpm_cm_dcis_nki.txt'))
tpm_long_format = expression_matrix.melt(id_vars=["Gene name", "Geneid"], var_name="sample_id", value_name="rnaseq_tpm").rename(columns={"Gene name": "symbol", "Geneid": "ensembl_gene_id"})
expression_template = read_metadata_with_fields(join(template, 'expression/expression_template-sheet.tsv'))
tpm_long_format = tpm_long_format.merge(submission_mms[['sample_id', 'platform_id']], on='sample_id', how='left')
for col in expression_template.columns:
    if col not in tpm_long_format.columns:
        tpm_long_format[col] = ''
tpm_long_format['platform_id'] = 'expression_' + tpm_long_format['platform_id']
for mid in submission_model_sample_dict.keys():
    temp = tpm_long_format[[sid in submission_model_sample_dict[mid] for sid in tpm_long_format['sample_id']]]
    temp[expression_template.columns].to_csv(join(NKI_submission, f'expression/NKI_expression_{mid}.tsv'), sep='\t', index=False)

    